In [1]:
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn
import numpy as np
#matplotlib.style.use('ggplot')
%matplotlib inline
matplotlib.rcParams['figure.figsize'] = (15.0, 5.0)

In [2]:
test_file_loc= 'Data/test.csv'
train_file_loc = 'Data/train.csv'

df_test_set = pd.DataFrame.from_csv(test_file_loc, index_col=None)
df_train_set = pd.DataFrame.from_csv(train_file_loc, index_col=None)

In [3]:
df_train_set.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [22]:
df_test_set.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


We see that the test set has one column less. 
Let's join the two datasets for now

In [31]:
total_df = df_train_set.drop(['Survived'], axis=1).append(df_test_set)
total_df.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [32]:
total_df.shape

(1309, 11)

# Feature Engineering

In [33]:
categorical_features = ['Embarked', 'Sex', 'Pclass'] 
ordinal_features = ['Age' ]
numerical_features = ['Parch', 'SibSp', 'Fare']
ignored_features = ['Ticket', 'Cabin', 'Name', 'Sex']
output_feature = ['Survived']
columns_to_plot = categorical_features + ordinal_features + numerical_features + output_feature

First let's start with some obvious things like one hot encoding 

In [34]:
def one_hot_encoding(categorical_variables, dataframe):
    """
    Function to convert categorical features to one-hot
    encoded features
    
    :param: categorical_variables list of categorical features
    :param: dataframe the dataframe to one-hot-encode
    :returns: pd.DataFrame()
    """
    return pd.get_dummies(dataframe, columns=categorical_variables, drop_first=True)

In [37]:
df_transformed = one_hot_encoding(categorical_features, total_df)
df_transformed.head()

,PassengerId,Name,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked_Q,Embarked_S,Sex_male,Pclass_2,Pclass_3
0,1,"Braund, Mr. Owen Harris",22.0,1,0,A/5 21171,7.2500,NaN,0,1,1,0,1
1,2,"Cumings, Mrs. John Bradley (Florence Briggs Th...",38.0,1,0,PC 17599,71.2833,C85,0,0,0,0,0
2,3,"Heikkinen, Miss. Laina",26.0,0,0,STON/O2. 3101282,7.9250,NaN,0,1,0,0,1
3,4,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",35.0,1,0,113803,53.1000,C123,0,1,0,0,0
4,5,"Allen, Mr. William Henry",35.0,0,0,373450,8.0500,NaN,0,1,1,0,1


In [38]:
df_transformed.describe()

,PassengerId,Age,SibSp,Parch,Fare,Embarked_Q,Embarked_S,Sex_male,Pclass_2,Pclass_3
count,1309.000000,1046.000000,1309.000000,1309.000000,1308.000000,1309.000000,1309.000000,1309.000000,1309.000000,1309.000000
mean,655.000000,29.881138,0.498854,0.385027,33.295479,0.093965,0.698243,0.644003,0.211612,0.541635
std,378.020061,14.413493,1.041658,0.865560,51.758668,0.291891,0.459196,0.478997,0.408607,0.498454
min,1.000000,0.170000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,328.000000,21.000000,0.000000,0.000000,7.895800,0.000000,0.000000,0.000000,0.000000,0.000000
50%,655.000000,28.000000,0.000000,0.000000,14.454200,0.000000,1.000000,1.000000,0.000000,1.000000
75%,982.000000,39.000000,1.000000,0.000000,31.275000,0.000000,1.000000,1.000000,0.000000,1.000000
max,1309.000000,80.000000,8.000000,9.000000,512.329200,1.000000,1.000000,1.000000,1.000000,1.000000


In [39]:
df_transformed.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1309 entries, 0 to 417
Data columns (total 13 columns):
PassengerId    1309 non-null int64
Name           1309 non-null object
Age            1046 non-null float64
SibSp          1309 non-null int64
Parch          1309 non-null int64
Ticket         1309 non-null object
Fare           1308 non-null float64
Cabin          295 non-null object
Embarked_Q     1309 non-null uint8
Embarked_S     1309 non-null uint8
Sex_male       1309 non-null uint8
Pclass_2       1309 non-null uint8
Pclass_3       1309 non-null uint8
dtypes: float64(2), int64(3), object(3), uint8(5)
memory usage: 98.4+ KB


We have missing values in the columns: age, Fare Cabin. So these are columns we can process. Other columns that we can can also process is names. Let's see why

In [44]:
df_transformed['Name'].head(20)

0                               Braund, Mr. Owen Harris
1     Cumings, Mrs. John Bradley (Florence Briggs Th...
2                                Heikkinen, Miss. Laina
3          Futrelle, Mrs. Jacques Heath (Lily May Peel)
4                              Allen, Mr. William Henry
5                                      Moran, Mr. James
6                               McCarthy, Mr. Timothy J
7                        Palsson, Master. Gosta Leonard
8     Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)
9                   Nasser, Mrs. Nicholas (Adele Achem)
10                      Sandstrom, Miss. Marguerite Rut
11                             Bonnell, Miss. Elizabeth
12                       Saundercock, Mr. William Henry
13                          Andersson, Mr. Anders Johan
14                 Vestrom, Miss. Hulda Amanda Adolfina
15                     Hewlett, Mrs. (Mary D Kingcome) 
16                                 Rice, Master. Eugene
17                         Williams, Mr. Charles

We notice that all the names have a title, i.e. "Mr", "Miss", etc. Let's extract these. 

In [ ]:
def process_titles(df):
    df['Title'] = df['Name'].map(lambda name: name.split(',')[1].split('.')[0].strip())
#     This yields in the following unique titles:
#     array(['Capt', 'Col', 'Don', 'Dona', 'Dr', 'Jonkheer', 'Lady', 'Major',
#        'Master', 'Miss', 'Mlle', 'Mme', 'Mr', 'Mrs', 'Ms', 'Rev', 'Sir',
#        'the Countess'], dtype=object)
#     We can further reduce this to: 
    title_mapping = {
        'Capt': 'Officer',
        'Col' : 'Officer',
        'Don' : 'Royalty',
        'Dona': 'Royalty',
        'Dr'  : 'Officer',
        'Jonkheer': 'Royalty',
        'Lady' : 'Royalty',
        'Major' : 'Officer',
        'Master': 'Master',
        'Miss' : 'Miss',
        'Mlle' : 'Miss',
        'Mme' : 'Mrs',
        'Mr' : 'Mr',
        'Mrs': 'Mrs',
        'Ms' : 'Ms',
        'Rev' : 'Officer',
        'Sir' : 'Royalty',
        'the Countess': 'Royalty'
    }
    df['Title'] = df.Title.map(title_mapping)
    return df
    
df_transformed = process_titles(df_transformed)
np.unique(df_transformed.Title)

In [63]:
df_transformed.head()

,PassengerId,Name,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked_Q,Embarked_S,Sex_male,Pclass_2,Pclass_3,Title
0,1,"Braund, Mr. Owen Harris",22.0,1,0,A/5 21171,7.2500,NaN,0,1,1,0,1,Mr
1,2,"Cumings, Mrs. John Bradley (Florence Briggs Th...",38.0,1,0,PC 17599,71.2833,C85,0,0,0,0,0,Mrs
2,3,"Heikkinen, Miss. Laina",26.0,0,0,STON/O2. 3101282,7.9250,NaN,0,1,0,0,1,Miss
3,4,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",35.0,1,0,113803,53.1000,C123,0,1,0,0,0,Mrs
4,5,"Allen, Mr. William Henry",35.0,0,0,373450,8.0500,NaN,0,1,1,0,1,Mr


Previously we have seen that age has been missing some values. Let's investigate how this feature looks like when plotted against other features like passenger class and sex 

In [68]:
total_df.groupby('Pclass').count()

,PassengerId,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
Pclass,,,,,,,,,,
1,323,323,323,284,323,323,323,323,256,321
2,277,277,277,261,277,277,277,277,23,277
3,709,709,709,501,709,709,709,708,16,709


In [69]:
total_df.groupby('Sex').count()

,PassengerId,Pclass,Name,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
Sex,,,,,,,,,,
female,466,466,466,388,466,466,466,466,141,464
male,843,843,843,658,843,843,843,842,154,843


Apparently it is mostly missing for passengers from class 2, and more for men than for women. 

The cabin column contains a lot of unknowns. But just throwing out this information might mean that we are throwing out valuable information. We can impute this with the letter "U" for unkown. 